In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sp
from time import time
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats.stats import pearsonr
import graphlab
from sklearn.cross_validation import train_test_split
from math import sqrt
%matplotlib inline

/home/anusha/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("processed_data.csv")

In [3]:
cold_users = df[df["user_review_count"] <=2]
cold_users.shape

(289628, 36)

In [17]:
grouped_rating = pd.DataFrame(cold_users.groupby('published_title')['rating'].mean())
grouped_rating['num of ratings'] = pd.DataFrame(cold_users.groupby('published_title')['rating'].count())
grouped_rating.sort_values('num of ratings',ascending=False, inplace=True)

In [18]:
grouped_rating.head()

,rating,num of ratings
published_title,,
the-web-developer-bootcamp,4.797765,8008
aws-certified-solutions-architect-associate,4.608980,7561
unitycourse,4.700592,6922
the-complete-web-developer-course-2,4.723423,6246
the-complete-guide-to-angular-2,4.741735,4900


In [20]:
course_mat = cold_users.pivot_table(index='userid',columns='published_title',values='rating')
course_mat.head()

published_title,1-day-mvp-find-the-right-idea-and-build-a-demo-in-1-day,1-hour-css,1-hour-html,1-hour-javascript,100k-blog-blueprint,101-crucial-lessons-they-dont-teach-you-in-business-mba-school,22-practical-lessons-in-leadership,30-days-of-python,30-minute-excel-formula-design,30-minute-outlook-training,...,work-from-home-setup-your-own-options-trading-business,writing-for-business,writing-mastery-creating-high-concept-ideas-that-sell,writing-mastery-productivity-hacks-for-writers,writing-with-flair-how-to-become-an-exceptional-writer,xamarin-forms-course,your-soul-is-calling,youtube-masterclass,youtubeforbusiness101,zero-cost-marketing-secrets-official-primer-course
userid,,,,,,,,,,,,,,,,,,,,,
1151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
webdev_user_ratings = course_mat['the-web-developer-bootcamp']
similar_to_webdev = course_mat.corrwith(webdev_user_ratings)
corr_webdev = pd.DataFrame(similar_to_webdev,columns=['correlation'])
corr_webdev.dropna(inplace=True)
corr_webdev.sort_values('correlation',ascending=False, inplace=True)
corr_webdev = corr_webdev.join(ratings['num of ratings'])
corr_webdev.reset_index(inplace=True)

In [36]:
corr_webdev[corr_webdev['num of ratings']>100].sort_values('correlation',ascending=False).head()

,published_title,correlation,num of ratings
0,forex-trading,1.0,693
6,meteor-react-tutorial,1.0,272
10,angular-2-and-nodejs-the-practical-guide,1.0,598
1,selenium-webdriver-with-java-testng-and-log4j,1.0,928
8,pixel-art-for-video-games,1.0,181


In [ ]:
corr_webdev.published_title